In [5]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from edu.javerianacali.create_dataset import CreateDataSet
from edu.javerianacali.model.ccn_model import ConvulationalNeuralNetwork
from edu.javerianacali.model.logistic_regression_model import LogisticRegressionModel
from edu.javerianacali.process_images import ProcessImages
from edu.javerianacali.data_aumentation import DataAugmentation
directorio = "/home/maucasco/Documents/maestria/proyecto_grado/pujc-advocato-filter-project-mngr/assets"


In [4]:

from edu.javerianacali.data_aumentation import DataAugmentation

DataAugmentation().augment_images(directorio+'poquitas'

)

/home/maucasco/Documents/maestria/proyecto_grado/pujc-advocato-filter-project-mngr/assets/poquitas/augmented_images


In [6]:

archivos = os.listdir(directorio)
for archivo in archivos:
    if archivo.endswith(".JPG") or archivo.endswith(".jpg"):
        ruta_imagen = os.path.join(directorio, archivo)
        titulos = ["Ori", "SinTex","Sinsomb", archivo]
        imagenes = ProcessImages().procesar_imagen(ruta_imagen,archivo,directorio)
        #ProcessImages().mostrar_imagenes(titulos, imagenes)



Type: <class 'numpy.ndarray'>
Shape of Image: (2564, 4000, 3)
Total Number of pixels: 30768000
Image data type: uint8
Dimension: 3
/home/maucasco/Documents/maestria/proyecto_grado/pujc-advocato-filter-project-mngr/assets/process/fil_P19_90 degrees.JPG
Type: <class 'numpy.ndarray'>
Shape of Image: (3900, 3000, 3)
Total Number of pixels: 35100000
Image data type: uint8
Dimension: 3
/home/maucasco/Documents/maestria/proyecto_grado/pujc-advocato-filter-project-mngr/assets/process/fil_Heilipus (20).jpg
Type: <class 'numpy.ndarray'>
Shape of Image: (3900, 3000, 3)
Total Number of pixels: 35100000
Image data type: uint8
Dimension: 3
/home/maucasco/Documents/maestria/proyecto_grado/pujc-advocato-filter-project-mngr/assets/process/fil_sano(221).jpg
Type: <class 'numpy.ndarray'>
Shape of Image: (2564, 4000, 3)
Total Number of pixels: 30768000
Image data type: uint8
Dimension: 3
/home/maucasco/Documents/maestria/proyecto_grado/pujc-advocato-filter-project-mngr/assets/process/fil_aug_P11_160 degre

In [ ]:
create_dataset = CreateDataSet().create_dataset(directorio)
#LogisticRegressionModel().create_dataset(directorio)
#LogisticRegressionModel().train_model(create_dataset[0], create_dataset[1])



In [ ]:
directorio = "/home/maucasco/Documents/maestria/proyecto_grado/pujc-advocato-filter-project-mngr/assets"
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from edu.javerianacali.create_dataset import CreateDataSet
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
import matplotlib.pyplot as plt
        # Crea un generador de datos de imagen con normalización
datagen = ImageDataGenerator(rescale=1./255)
print(directorio
+'/tensor')
# Carga las imágenes desde el directorio
# Asegúrate de que dentro de este directorio, las imágenes estén organizadas en subdirectorios según su etiqueta/clase
generator = datagen.flow_from_directory(
directorio,
target_size=(100, 130),  # Asegúrate de que estas dimensiones coincidan con el preprocesamiento que hiciste
batch_size=180,
class_mode='categorical' ) # Usa 'binary' si solo tienes dos clases

In [ ]:
from keras.layers import Dropout
from keras.optimizers import Adam

modelo = Sequential([
    Conv2D(32, (5,5), activation='relu', input_shape=(100, 130, 3)),
    MaxPooling2D(pool_size=(2,2)),
    Conv2D(64, (5,5), activation='relu'),
    MaxPooling2D(pool_size=(2,2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),  # Capa de abandono para regularización
    Dense(5, activation='softmax')
])
modelo.compile(optimizer=Adam(learning_rate=0.001),  # Ajustar la tasa de aprendizaje
    loss='categorical_crossentropy',  
    metrics=['accuracy'])
historial = modelo.fit(
    generator,
    epochs=10,  # Aumentar el número de épocas
    steps_per_epoch=generator.samples // 180
)

plt.plot(historial.history['accuracy'])
plt.title('Precisión del modelo')
plt.ylabel('Precisión')
plt.xlabel('Época')
plt.legend(['Entrenamiento'], loc='upper left')
plt.show()

# Gráfica de pérdida
plt.plot(historial.history['loss'])
plt.title('Pérdida del modelo')
plt.ylabel('Pérdida')
plt.xlabel('Época')
plt.legend(['Entrenamiento'], loc='upper left')
plt.show()

In [ ]:
from keras.models import Model
import matplotlib.pyplot as plt

# Crear un modelo que devuelva las activaciones de la primera capa convolucional
activation_model = Model(inputs=modelo.input, outputs=modelo.layers[0].output)

# Obtener las activaciones de la primera capa convolucional para la primera imagen en el conjunto de entrenamiento
activations = activation_model.predict(generator[0][0][0].reshape(1, 100, 130, 3))

# Visualizar las activaciones de los primeros 6 filtros
fig, axes = plt.subplots(nrows=2, ncols=3, figsize=(15, 10))
for i, ax in enumerate(axes.flat):
    ax.imshow(activations[0, :, :, i], cmap='viridis')
    ax.axis('off')
plt.show()